## EEG and KAN for HMS Brain Classificatio 

### CELL 1: COMPLETE SETUP (GPU, SEED, DATA, DATASET)

In [1]:
import os
from pathlib import Path
import random
import time
import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

%matplotlib inline

print("[SETUP] Loading libraries... ✓\n")

# ===== GPU SETUP =====
print("[DEVICE] Checking GPU...")
print(f"  CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda:0")
    torch.cuda.empty_cache()
    print("  Using: GPU ✓")
else:
    device = torch.device("cpu")
    print("  Using: CPU")

# ===== REPRODUCIBILITY =====
print("\n[SEED] Setting random seeds...")

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)
print("  Random seed: 42 ✓")

# ===== PATHS =====
print("\n[PATHS] Checking data directories...")
DATA_PKG = Path("data_package")
SPEC_DIR = Path("spec_hr_out")

assert DATA_PKG.exists(), "❌ data_package/ not found"
assert SPEC_DIR.exists(), "❌ spec_hr_out/ not found"
print(f"  {DATA_PKG} ✓")
print(f"  {SPEC_DIR} ✓")

print("\n" + "="*60)
print("SETUP COMPLETE ✓")
print("="*60)

# ===== LOAD DATA =====
print("\n[DATA] Loading metadata and labels...\n")

meta_use = pd.read_csv(DATA_PKG / "meta_use.csv")
print(f"  Meta: {meta_use.shape}")

lbl = np.load(DATA_PKG / "labels.npz", allow_pickle=True)
y_soft  = lbl["y_soft"]
w_conf  = lbl["w_conf"]
classes = [c for c in lbl["classes"]]

print(f"  Labels: {y_soft.shape}")
print(f"  Classes: {classes}")

# Class distribution
y_hard = y_soft.argmax(axis=1)
print("\n  Class distribution:")
for i, cls in enumerate(classes):
    count = (y_hard == i).sum()
    print(f"    {cls:10s}: {count:5d} ({100*count/len(y_hard):5.1f}%)")

# ===== CREATE FOLDS =====
print("\n[FOLDS] Creating stratified folds...")
N_SPLIT = 5
skf = StratifiedKFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
folds = list(skf.split(meta_use, y_hard))
print(f"  {len(folds)} folds created")
print(f"  Fold 0: train={len(folds[0][0])}, val={len(folds[0][1])}")

print("\n" + "="*60)
print("DATA LOADED ✓")
print("="*60)

# ===== DATASET CLASS =====
print("\n[DATASET] Creating dataset class...\n")

class SpecDataset(Dataset):
    """EEG Spectrogram Dataset"""
    
    def __init__(self, df, root_dir, y_soft, w_conf, F_target=81, T_target=600):
        self.df = df.reset_index(drop=True)
        self.root = Path(root_dir)
        self.y_soft = y_soft
        self.w_conf = w_conf
        self.F_target = F_target
        self.T_target = T_target

    def __len__(self):
        return len(self.df)

    def _center_crop_pad(self, x):
        C, F, T = x.shape
        # Frequency
        if F >= self.F_target:
            f0 = (F - self.F_target) // 2
            x = x[:, f0:f0+self.F_target, :]
        else:
            pad = self.F_target - F
            x = np.pad(x, ((0,0),(pad//2, pad-pad//2),(0,0)), mode="constant")
        # Time
        if T >= self.T_target:
            t0 = (T - self.T_target) // 2
            x = x[:, :, t0:t0+self.T_target]
        else:
            pad = self.T_target - T
            x = np.pad(x, ((0,0),(0,0),(pad//2, pad-pad//2)), mode="constant")
        return x.copy()

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        eid = int(row.eeg_id)
        npz = np.load(self.root / f"{eid}_hr.npz")
        x = npz["x"]
        x = self._center_crop_pad(x)
        x = torch.from_numpy(x).float()
        # Resize to 224x224
        x = F.interpolate(x.unsqueeze(0), size=(224,224),
                          mode="bilinear", align_corners=False).squeeze(0)
        y = torch.from_numpy(self.y_soft[self.df.index[idx]]).float()
        w = torch.tensor(self.w_conf[self.df.index[idx]], dtype=torch.float32)
        return x, y, w

# Test dataset
tr_idx, va_idx = folds[0]
ds_test = SpecDataset(meta_use.iloc[tr_idx[:50]], SPEC_DIR, y_soft, w_conf)
print(f"  Test dataset: {len(ds_test)} samples")

dl_test = DataLoader(ds_test, batch_size=4, shuffle=True, num_workers=0, pin_memory=False)
xb, yb, wb = next(iter(dl_test))
print(f"  Test batch: x={xb.shape}, y={yb.shape}, w={wb.shape}")

del dl_test, ds_test, xb, yb, wb
gc.collect()

print("\n" + "="*60)
print("DATASET READY ✓")
print("="*60)

print("\n" + "="*60)
print(" ✅ ALL SETUP COMPLETE - READY TO TRAIN! ".center(60, "="))
print("="*60)

[SETUP] Loading libraries... ✓

[DEVICE] Checking GPU...
  CUDA available: True
  GPU: NVIDIA GeForce RTX 5060 Ti
  Using: GPU ✓

[SEED] Setting random seeds...
  Random seed: 42 ✓

[PATHS] Checking data directories...
  data_package ✓
  spec_hr_out ✓

SETUP COMPLETE ✓

[DATA] Loading metadata and labels...

  Meta: (17089, 3)
  Labels: (17089, 6)
  Classes: ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

  Class distribution:
    seizure   :  2716 ( 15.9%)
    lpd       :  2583 ( 15.1%)
    gpd       :  1814 ( 10.6%)
    lrda      :   936 (  5.5%)
    grda      :  1835 ( 10.7%)
    other     :  7205 ( 42.2%)

[FOLDS] Creating stratified folds...
  5 folds created
  Fold 0: train=13671, val=3418

DATA LOADED ✓

[DATASET] Creating dataset class...

  Test dataset: 50 samples
  Test batch: x=torch.Size([4, 4, 224, 224]), y=torch.Size([4, 6]), w=torch.Size([4])

DATASET READY ✓

========== ✅ ALL SETUP COMPLETE - READY TO TRAIN! ==========


### CELL 2:EEGNet

In [7]:
class EEGNet_4Ch(nn.Module):
    """EEGNet for 4-channel EEG spectrograms - FIXED"""
    
    def __init__(self, n_classes=6, n_channels=4, F1=8, F2=16, D=2, dropout=0.25, pretrained=False):
        super().__init__()
        
        # Block 1: Temporal convolution
        self.block1 = nn.Sequential(
            nn.Conv2d(n_channels, F1, (1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(F1),
        )
        
        # Block 2: Depthwise spatial convolution
        self.block2 = nn.Sequential(
            nn.Conv2d(F1, F1 * D, (n_channels, 1), groups=F1, bias=False),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(dropout)
        )
        
        # Block 3: Separable convolution
        self.block3 = nn.Sequential(
            nn.Conv2d(F1 * D, F1 * D, (1, 16), padding=(0, 8), groups=F1 * D, bias=False),
            nn.Conv2d(F1 * D, F2, (1, 1), bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(dropout)
        )
        
        # ✅ FIX: Adaptive pooling ensures consistent size!
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier - now just F2 features
        self.classifier = nn.Linear(F2, n_classes)  # 16 -> 6
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.block1(x)           # [B, 8, 224, 224]
        x = self.block2(x)           # [B, 16, 1, 56]
        x = self.block3(x)           # [B, 16, 1, ?]
        x = self.adaptive_pool(x)    # [B, 16, 1, 1] ✅ FIXED!
        x = x.view(x.size(0), -1)    # [B, 16]
        return self.classifier(x)    # [B, 6]

print("✅ EEGNet FIXED!")

✅ EEGNet FIXED!


### CELL 3: KAN ARCHITECTURE

In [8]:
class KANLayer(nn.Module):
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3):
        super().__init__()
        self.coefficients = nn.Parameter(torch.randn(out_features, in_features, grid_size) * 0.1)
        self.base_weight = nn.Parameter(torch.randn(out_features, in_features) * 0.1)
        self.register_buffer('grid', torch.linspace(-1, 1, grid_size))
        self.spline_order = spline_order
    
    def b_spline_basis(self, x):
        x = x.unsqueeze(-1)
        grid = self.grid.view(1, 1, -1)
        distances = torch.abs(x - grid)
        basis = torch.relu(1 - distances) ** self.spline_order
        return basis / (basis.sum(dim=-1, keepdim=True) + 1e-8)
    
    def forward(self, x):
        basis = self.b_spline_basis(x)
        return torch.einsum('bik,oik->bo', basis, self.coefficients) + F.linear(x, self.base_weight)

class KAN_4Ch(nn.Module):
    def __init__(self, n_classes=6, n_channels=4, grid_size=5, pretrained=False):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(n_channels, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.kan1 = KANLayer(256, 128, grid_size)
        self.dropout1 = nn.Dropout(0.5)
        self.kan2 = KANLayer(128, 64, grid_size)
        self.dropout2 = nn.Dropout(0.3)
        self.output = nn.Linear(64, n_classes)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight) if isinstance(m, nn.Conv2d) else nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.features(x).view(x.size(0), -1)
        x = self.dropout1(self.kan1(x))
        x = self.dropout2(self.kan2(x))
        return self.output(x)

print("✅ KAN defined")

✅ KAN defined


### CELL4:Training Components

In [9]:
class SoftFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=3.0):
        super().__init__()
        self.alpha, self.gamma = alpha, gamma
    
    def forward(self, logits, soft_targets, sample_weights=None):
        hard_targets = soft_targets.argmax(dim=1)
        probs = F.softmax(logits, dim=1)
        p_t = probs.gather(1, hard_targets.unsqueeze(1)).squeeze(1)
        loss = ((1 - p_t) ** self.gamma) * (-(soft_targets * F.log_softmax(logits, dim=1)).sum(dim=1))
        if self.alpha is not None:
            loss = self.alpha[hard_targets] * loss
        if sample_weights is not None:
            loss = loss * sample_weights
        return loss.mean()

def create_hybrid_loader(fold=0, target_ratio=0.4, weight_power=3.0, batch_size=16):
    tr_idx, va_idx = folds[fold]
    df_tr = meta_use.iloc[tr_idx]
    y_soft_tr, w_conf_tr = y_soft[tr_idx], w_conf[tr_idx]
    
    y_hard = y_soft_tr.argmax(axis=1)
    counts = np.bincount(y_hard, minlength=6)
    target = int(counts.max() * target_ratio)
    
    indices_add = []
    for i in range(6):
        mask = y_hard == i
        if mask.sum() < target:
            idx = np.where(mask)[0]
            indices_add.extend(np.random.choice(idx, target - mask.sum(), True))
    
    all_idx = np.concatenate([np.arange(len(y_hard)), indices_add])
    np.random.shuffle(all_idx)
    
    df_tr_over = df_tr.iloc[all_idx].reset_index(drop=True)
    y_soft_over, w_conf_over = y_soft_tr[all_idx], w_conf_tr[all_idx]
    
    y_hard_over = y_soft_over.argmax(axis=1)
    counts_over = np.bincount(y_hard_over, minlength=6)
    weights = (len(y_hard_over) / (counts_over + 1)) ** weight_power
    weights = torch.FloatTensor(weights / weights.sum() * 6)
    
    sampler = WeightedRandomSampler(weights[y_hard_over].numpy(), len(y_hard_over), True)
    
    ds_tr = SpecDataset(df_tr_over, SPEC_DIR, y_soft_over, w_conf_over)
    dl_tr = DataLoader(ds_tr, batch_size, sampler=sampler, num_workers=0)
    
    ds_va = SpecDataset(meta_use.iloc[va_idx], SPEC_DIR, y_soft[va_idx], w_conf[va_idx])
    dl_va = DataLoader(ds_va, batch_size, shuffle=False, num_workers=0)
    
    return dl_tr, dl_va, weights

@torch.no_grad()
def evaluate_full(model, loader):
    model.eval()
    preds, targets = [], []
    for x, y, w in loader:
        preds.append(model(x.to(device)).argmax(1).cpu().numpy())
        targets.append(y.argmax(1).cpu().numpy())
    y_pred, y_true = np.concatenate(preds), np.concatenate(targets)
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall_macro': recall_score(y_true, y_pred, average='macro', zero_division=0),
        'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0)
    }

print("✅ Training components ready")

✅ Training components ready


### CELL 5:# CELL 5: TRAINING FUNCTION

In [10]:
def train_model(model_name, model_class, fold=0, epochs=30, batch_size=16, base_lr=3e-4, gamma=3.0, patience=5):
    print("\n" + "="*80)
    print(f" TRAINING {model_name} ".center(80, "="))
    print("="*80)
    
    print("\n[1/5] Creating dataloaders...")
    dl_tr, dl_va, class_weights = create_hybrid_loader(fold, batch_size=batch_size)
    print(f"✓ Train: {len(dl_tr.dataset)}, Valid: {len(dl_va.dataset)}")
    
    print("\n[2/5] Creating model...")
    model = model_class(n_classes=6, pretrained=False).to(device)
    print(f"✓ Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
    
    print(f"\n[3/5] Setup (Focal gamma={gamma})...")
    criterion = SoftFocalLoss(class_weights.to(device), gamma)
    optimizer = torch.optim.AdamW(model.parameters(), base_lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    print("\n[4/5] Training...")
    print(f"{'Ep':<4s} {'Loss':>8s} {'Acc':>8s} {'Prec':>8s} {'Rec':>8s} {'F1':>8s} {'Time':>6s} {'Status':<10s}")
    print("-"*80)
    
    best_f1, best_state, no_improve = 0.0, None, 0
    
    for epoch in range(1, epochs + 1):
        t0 = time.time()
        
        model.train()
        train_loss, n = 0.0, 0
        for x, y, w in dl_tr:
            x, y, w = x.to(device), y.to(device), w.to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y, w)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item() * x.size(0)
            n += x.size(0)
        
        train_loss /= n
        val_results = evaluate_full(model, dl_va)
        scheduler.step()
        
        if val_results['f1_macro'] > best_f1:
            best_f1 = val_results['f1_macro']
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
            status = "✓ BEST"
        else:
            no_improve += 1
            status = f"({no_improve}/{patience})"
        
        print(f"{epoch:02d}  {train_loss:8.4f} {val_results['accuracy']:8.4f} {val_results['precision_macro']:8.4f} "
              f"{val_results['recall_macro']:8.4f} {val_results['f1_macro']:8.4f} {time.time()-t0:5.0f}s  {status}")
        
        if no_improve >= patience:
            print(f"\nEarly stop at epoch {epoch}")
            break
    
    if best_state:
        model.load_state_dict(best_state)
    
    print("\n[5/5] Final evaluation...")
    final = evaluate_full(model, dl_va)
    
    print("\n" + "="*80)
    print(f" {model_name} RESULTS ".center(80, "="))
    print("="*80)
    print(f"Accuracy:  {final['accuracy']:.4f}")
    print(f"Precision: {final['precision_macro']:.4f}")
    print(f"Recall:    {final['recall_macro']:.4f}")
    print(f"F1 Score:  {final['f1_macro']:.4f}")
    print("="*80)
    
    safe_name = model_name.lower().replace(" ", "_").replace("-", "")
    torch.save(model.state_dict(), f'{safe_name}_best.pth')
    print(f"\n💾 Saved: {safe_name}_best.pth")
    
    return model, final

print("✅ Training function ready")

✅ Training function ready


## Train Models

In [11]:
results = {}
model_eegnet, results['EEGNet'] = train_model('EEGNet', EEGNet_4Ch, epochs=30, batch_size=16)


=============================== TRAINING EEGNet ================================

[1/5] Creating dataloaders...
✓ Train: 17289, Valid: 3418

[2/5] Creating model...
✓ Parameters: 0.00M

[3/5] Setup (Focal gamma=3.0)...

[4/5] Training...
Ep       Loss      Acc     Prec      Rec       F1   Time Status    
--------------------------------------------------------------------------------
01    1.1628   0.1811   0.2208   0.2662   0.1525   553s  ✓ BEST
02    1.0803   0.2025   0.2240   0.2928   0.1888   112s  ✓ BEST
03    1.0311   0.2367   0.2063   0.3452   0.2393    69s  ✓ BEST
04    0.9984   0.2238   0.2199   0.3448   0.2380    61s  (1/5)
05    0.9900   0.2589   0.2167   0.3667   0.2628    58s  ✓ BEST
06    0.9734   0.2575   0.2223   0.3741   0.2702    54s  ✓ BEST
07    0.9387   0.2402   0.2302   0.3708   0.2632    57s  (1/5)
08    0.9337   0.2618   0.2435   0.3853   0.2843    52s  ✓ BEST
09    0.9259   0.2659   0.2207   0.3623   0.2720    49s  (1/5)
10    0.9259   0.2923   0.2429   0.4110

In [12]:
model_kan, results['KAN'] = train_model('KAN', KAN_4Ch, epochs=30, batch_size=16)


================================= TRAINING KAN =================================

[1/5] Creating dataloaders...
✓ Train: 17289, Valid: 3418

[2/5] Creating model...
✓ Parameters: 0.64M

[3/5] Setup (Focal gamma=3.0)...

[4/5] Training...
Ep       Loss      Acc     Prec      Rec       F1   Time Status    
--------------------------------------------------------------------------------
01    2.2846   0.1878   0.2778   0.3170   0.2170    98s  ✓ BEST
02    1.4887   0.2420   0.2912   0.3655   0.2553    61s  ✓ BEST
03    1.1565   0.2689   0.3387   0.4115   0.2800    58s  ✓ BEST
04    0.9416   0.3031   0.2744   0.4489   0.3098   133s  ✓ BEST
05    0.8355   0.3189   0.2893   0.4649   0.3327    72s  ✓ BEST
06    0.7712   0.3344   0.3331   0.5061   0.3645    62s  ✓ BEST
07    0.7204   0.2616   0.3292   0.4177   0.2982    59s  (1/5)
08    0.6937   0.3607   0.3153   0.5091   0.3726    59s  ✓ BEST
09    0.6623   0.3069   0.3379   0.4654   0.3288    58s  (1/5)
10    0.6432   0.3444   0.2750   0.466